# Advanced Fine-Tuning for Novel Writing
Enhanced training with multi-epoch, evaluation, checkpointing, and early stopping.

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" "trl<0.8.0" peft accelerate bitsandbytes
!pip install wandb  # For experiment tracking


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 8192
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load full dataset
full_dataset = load_dataset("json", data_files="train.jsonl", split="train")

# Split into train/validation (90/10)
train_data = full_dataset.train_test_split(test_size=0.1)
train_dataset = train_data["train"]
eval_dataset = train_data["test"]

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# Format both datasets
train_dataset = train_dataset.map(formatting_prompts_func, batched = True)
eval_dataset = eval_dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
import wandb

# Optional: Track with Weights & Biases
# wandb.init(project="novel-writer", name="llama3-finetune")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        output_dir = "checkpoints",
        num_train_epochs = 3,  # Multi-epoch training
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        eval_strategy = "steps",  # Evaluate during training
        eval_steps = 50,
        save_strategy = "steps",  # Save checkpoints
        save_steps = 50,
        save_total_limit = 3,  # Keep only last 3 checkpoints
        load_best_model_at_end = True,  # Load best checkpoint
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        report_to = "wandb",  # Track with W&B
        run_name = "llama3-novel-finetune",
        seed = 3407,
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop if no improvement
)

trainer.train()

In [ ]:
# Save final model
model.save_pretrained("lora_model_final")
tokenizer.save_pretrained("lora_model_final")

# Merge LoRA weights (optional, for deployment)
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model")